# This is a work-in-progress notebook

We wish to know this:

1. How well does the model identify the correct number of senses for the target word?
2. **How well does the model identify the correct senses for the target word?**
3. **How well does the model assign the right words to a given sense of the target word?**
4. How well does the model assign the senses to the time intervals for the target word?

The script will evaluate **Q2** and **Q3**. Q4 will follow.

In [1]:
# Basic variables and imports:

import codecs, csv, os, time, re, io
from os import listdir
from os.path import isfile, join

# directories
dir_in = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "evaluation", "evaluation_input"))
dir_out = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "evaluation", "evaluation_output"))

s_senses = io.open(dir_in+"/senses_69419.txt","r")
k_senses = io.open(dir_in+"/mus.dat","r")

file_senses = s_senses.readlines()[1:]
output_senses = k_senses.read()

_io.TextIOWrapper

## TODO

- ~~create the notebook~~
- ~~organise the notebook~~
- ~~write "general idea" pseudocode for the evaluation~~
- ~~get input files~~
- figure out data structures to store the variables
- write actual code

## Q2: How well does the model identify the correct senses for the target word?

### Pseudocode 

In [ ]:
# For each target word, we have a list of senses  s (given by the expert)
# For each target word, we have a list of senses k (given by the model)
# This Q consists in matching s and k, and doing so in a confident way --> confidence score

for each k:
    for each s:
        create conf(k,s)

# What is conf(k,s)?
        conf(k,s) = (p1*match(w1,s)+p2*match(w1,s)+px(wx,s))/10 WHERE
    
            px = probability of word wx 
                
                and
            
            match(wx,s) =   1/number_of_senses_assigned_to_wx if s_is_one_of_them 
            
                    or 
                            0 if w_is_not_associated_to_s
                
# Once we have gone through all s for one k, we have to choose the best k for s. How? (TBD, cfr Valerio and Barbara)

# Once all ks have been assigned to all ss (or NA), we can calculate a general confidence score for the model.
# One easy way to do that: 

conf_score_model = number_of_non_NA/k

### Real code

Steps:

- extract all senses from the file
- use those senses as keys for a dictionary, `dict_of_words`
- fill the dictionary: for each key, we store a list of words pertaining to that sense
- transform the lists as sets so as to remove duplicates within the same sense
- create a dictionary with a word as a key and its weight as a value, depending on how many senses it appears
- parse the model output and get the probability weights for each word

Todo:

- do not take into account the first line
- take care of empty lines


In [8]:
expert_senses = list() # list where we store all sense ids provided by expert
for s in file_senses: # 60 for testing purposes
    s = s.split("\t")
    sense = s[11] # The sense ID is after the 10th tab
    expert_senses.append(sense)
    
#print(len(expert_senses),expert_senses,len(set(expert_senses)))

expert_senses = list(set(expert_senses)) # we only keep the unique senses
number_of_s = len(expert_senses)  # we create a variable that stores the number of unique senses
print("Number of senses:",number_of_s)

# This dictionary has a sense as a key, and a list of words as a value. 
dict_of_words = dict()
# This list stores all words
list_of_all_words = list()
# This dictionary stores all words as keys and their weight as value
word_weight = dict()

for i in range(0,number_of_s): # for each sense, we create a dictionary entry which has a list as value
    dict_of_words[expert_senses[i]] = list()

    for s in file_senses: # we go back in the file
        s = s.split("\t") # splitting on tabs
        
        sentence_of_ids = s[8] # 8 is for IDs, 9 is for words
        list_of_ids = sentence_of_ids.split(" ")  # splitting on spaces
        for word_id in list_of_ids:
            if s[11] == expert_senses[i]:      # we store all words for one sense 
                dict_of_words[expert_senses[i]].append(word_id)
            list_of_all_words.append(word_id) # we store all words, we'll iterate over that for scores
        

    # Here, we remove duplicates
    #dict_of_words[expert_senses[i]].append("79223") #testing
    dict_of_words[expert_senses[i]] = set(dict_of_words[expert_senses[i]]) 
      
    print(expert_senses[i],set(dict_of_words[expert_senses[i]]))
    print("\n\n")


Number of senses: 4
w {'61073', '71312', '16400', '260', '32657', '35541', '1379', '26447', '71976', '56607', '61708', '58444', '11072', '86305', '12252', '64985', '12625', '104421', '1083', '46804', '113060', '48549', '112350', '46178', '76801', '31333', 'nlsj56179', '65747', '114558', '16132', '109757', '75552', '35581', '104374', '11058', '116416', '39186', '53826', '19282', '24436', '96193', '69419', '84410', '3398', '31236', '62204', '71308', '1989', '86112', '31562', '41357', '65552', 'nlsj32160', '98747', '43260', '98541', '23903', '293', '65295', '34522', '49589', '33631', '78297', '31537', '21920', '61016', '4548', '23658', '114753', '63845', '1984', '45996', '90410', '24340', '12210', '28592', '26886', 'nlsj7765', '23799', '30790', 'nlsj5856', '5390', '79223', '96106', 'nlsj4603', '63772', '44552', 'nlsj5105', '54356', '2219', '46195', '73064', '38714', '1377', '1093', '85306', '92927', '4958', '70958', '67132', 'nlsj7011', '35642', '115213', '42214', '103701', '28569', '1044

In [9]:
# For every word in the list of words that we have
# we count the number of senses it appears in
# we use that number to divide its importance: 1 sense = 1 importance; 2 senses = 0.5 importance
# this can be finetuned

for word in list_of_all_words:
    x = 0
    for i in range(0,number_of_s):
        if word in dict_of_words[expert_senses[i]]:
            x += 1 
        if x != 0:
            word_weight[word] = 1/x
        
    
    print(word,word_weight[word])  

36165 1.0
18334 0.5
34855 1.0
41633 1.0
71314 1.0
69419 0.25
15380 1.0
42890 1.0
110302 1.0
104311 1.0
16400 0.5
76184 1.0
109730 1.0
110606 1.0
22997 1.0
83253 1.0
85306 0.3333333333333333
71308 0.5
19546 1.0
51241 0.5
50093 1.0
47665 1.0
63143 1.0
45996 0.25
113560 0.5
69419 0.25
62204 0.3333333333333333
22739 1.0
84534 0.5
22209 0.5
103012 1.0
23468 1.0
13579 1.0
45996 0.25
95258 1.0
19546 1.0
64914 1.0
80557 1.0
57262 0.5
29828 1.0
23468 1.0
110598 1.0
75263 1.0
69419 0.25
19641 0.5
nlsj78558 1.0
80761 1.0
23468 1.0
71308 0.5
22502 1.0
95258 1.0
26447 0.5
22100 1.0
106502 0.5
30911 1.0
69419 0.25
22100 1.0
22209 0.5
53442 1.0
8909 0.5
nlsj52509 1.0
19641 0.5
106502 0.5
75263 1.0
nlsj71743 0.5
37776 1.0
50824 1.0
nlsj78558 1.0
4335 1.0
77928 1.0
100693 1.0
22502 1.0
8665 0.5
41918 0.5
31236 0.5
64448 1.0
51661 1.0
30911 1.0
29828 1.0
62204 0.3333333333333333
75263 1.0
22209 0.5
7177 1.0
75954 1.0
82665 1.0
69419 0.25
22209 0.5
22100 1.0
53956 1.0
19641 0.5
22209 0.5
63314 1.0
31491 

#### parsing output.dat
- split on "===============  per time  ===============" and keep first part
- transform that into a list, then
- get lines that start with "p(w|s)"
- count those, k = that number
- split the line on ":", keep the second part
- split the rest on ";", it's [ID] = prob_from_this_ID

In [10]:
lines_output = output_senses.split("===============  per time  ===============")[0].split("\n")

number_of_the_k = 0

k_words_with_prob = dict()

for line in lines_output:
    if line[:6] == "p(w|s)":
        line = line.split(":")[1]
        line = line.split(";")
        print(number_of_the_k,line)
        dico_word_prob = dict()
        k_words_with_prob[number_of_the_k] = list()
        
        line = line[:-1] # last item of the list is empty
        for word_prob in line:
        
            word_prob = word_prob.split(",")
            for word in word_prob:
                matchObj = re.findall("([\d.\w]*)",word)
                if matchObj:
                    
                    matchObj = list(filter(None,matchObj))
                    
            
            print("word id",matchObj[0],"; probability",matchObj[1])
            dico_word_prob[matchObj[0]] = matchObj[1]
        k_words_with_prob[number_of_the_k] = dico_word_prob

        number_of_the_k += 1
        



0 ['  28355 (0.088) ', ' 69419 (0.069) ', ' 57460 (0.056) ', ' 114587 (0.042) ', ' 42071 (0.041) ', ' 35267 (0.035) ', ' 51647 (0.035) ', ' 64448 (0.023) ', ' 45980 (0.018) ', ' 53826 (0.017) ', ' ']
word id 28355 ; probability 0.088
word id 69419 ; probability 0.069
word id 57460 ; probability 0.056
word id 114587 ; probability 0.042
word id 42071 ; probability 0.041
word id 35267 ; probability 0.035
word id 51647 ; probability 0.035
word id 64448 ; probability 0.023
word id 45980 ; probability 0.018
word id 53826 ; probability 0.017
1 ['  79223 (0.063) ', ' 92927 (0.057) ', ' 46574 (0.038) ', ' 67660 (0.038) ', ' 103085 (0.036) ', ' 86112 (0.030) ', ' 101982 (0.029) ', ' 75808 (0.026) ', ' 68539 (0.024) ', ' 54607 (0.024) ', ' ']
word id 79223 ; probability 0.063
word id 92927 ; probability 0.057
word id 46574 ; probability 0.038
word id 67660 ; probability 0.038
word id 103085 ; probability 0.036
word id 86112 ; probability 0.030
word id 101982 ; probability 0.029
word id 75808 ; pr

#### k_words_with_prob
This dictionary has the sense number 'k' as keys and the a dictionary of [word] = probability as values.
Example below.

In [197]:
print("Probability for word ID 5390 in sense k = 4:",k_words_with_prob[4]["5390"])

Probability for word ID 5390 in sense k = 4: 0.019


In [19]:
for key in k_words_with_prob.keys():
    print("output sense",key)
    for i in range(0,number_of_s):
        print("\texpert sense number ", i, expert_senses[i])
        for second_key in k_words_with_prob[key].keys(): # Barbara's note: shouldn't it be k_words_with_prob[i] here?
            print("\t\tword from annotation for sense", key, ":", second_key)
            if second_key in dict_of_words[expert_senses[i]]:
                print("\t\t\tword ", second_key, "is in output for sense", key, "with probability:", k_words_with_prob[key][second_key])
# Here we get all the senses and for each sense we do a matching between the k words and s words and get the probability
# For some reason the first word for each sense arrives several times


output sense 0
	expert sense number  0 w
		word from annotation for sense 0 : 28355
		word from annotation for sense 0 : 69419
			word  69419 is in output for sense 0 with probability: 0.069
		word from annotation for sense 0 : 57460
		word from annotation for sense 0 : 114587
		word from annotation for sense 0 : 42071
		word from annotation for sense 0 : 35267
		word from annotation for sense 0 : 51647
		word from annotation for sense 0 : 64448
		word from annotation for sense 0 : 45980
		word from annotation for sense 0 : 53826
			word  53826 is in output for sense 0 with probability: 0.017
	expert sense number  1 mus-1
		word from annotation for sense 0 : 28355
			word  28355 is in output for sense 0 with probability: 0.088
		word from annotation for sense 0 : 69419
			word  69419 is in output for sense 0 with probability: 0.069
		word from annotation for sense 0 : 57460
		word from annotation for sense 0 : 114587
			word  114587 is in output for sense 0 with probability: 0.042
		wo

Barbara's note: "key" and "i" may be different senses; for example, for key=0, this is the first sense in the output, and i=0 is the first sense annotated by the expert. I think what we want here is to try matching all key values with all i values; for each (key, i) pair, we get the conf(key, i), as in pages 5 ff. of the Goals and plan.docx document.

Then, when possible, we can pick the best "i" for each "key"; we haven't yet decided how, but it will probably have to do with the maximum conf value.

Once we have a key-->i mapping, we can calculate precision and recall..

## Q3: How well does the model assign the right words to a given sense of the target word?

In [ ]:
# For each k, we use the words given by the expert as unquestionable truth.
# Judging the model's assignment of words to a given sense becomes a question of precision and recall.

# precision is all correct w weighted by their respective probabilities / all w weighted by their probabilities
for each k:
    for each w:
        if w in expert_list:
            w_weight = p*1
            numerator += w_weight
        w_weight = p*1
        denominator += w_weight
    precision = numerator/denominator
    
# recall is all correct w weighted by their respective probabilities / all w assigned to the sense by the expert
for each k:
    for each w:
        if w in expert_list:
            w_weight = p*1
            numerator += w_weight
    denominator = len(expert_list)
    recall = numerator/denominator
    
# f-score can be used as well

for each k:
    f_score = 2 * precision * recall / (precision+recall)